Lets finally put the pieces together and build a classifier for images with ovals and rectangles.

In [ ]:
import shutil
shutil.rmtree("examplemodel")

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import sys, re

W = 600
H = 600
D = 1

input_images = []
output_classes = []

def loadImages(img_dir):
    global input_images
    global output_classes
    global W
    global H
    input_images = []
    output_classes = []
    print("Loading images")
    for imgFilename in os.listdir(img_dir):
        imgPath = os.path.join(img_dir, imgFilename)
        if not imgPath.endswith(".jpg"):
            continue
        a = cv2.imread(imgPath,0)#1=color, 0=grayscale
        if a is None:
            continue
        H, W = a.shape[:2]
        if "rect" not in imgFilename:
            input_images.append(a.flatten())
            output_classes.append(0)
        else:
            input_images.append(a.flatten())
            output_classes.append(1)
    input_images  = np.asarray(input_images, dtype=np.float32)
    output_classes = np.asarray(output_classes, dtype=np.int32)
    print(str(output_classes))

Model... add some layers like tf.layers.conv2d, tf.layers.max_pooling2d, tf.layers.dense

#Example layer prototypes
<br>conv    = tf.layers.conv2d(input_layer, filters=, kernel_size=, padding="same", activation=tf.nn.relu)
<br>flat    = tf.reshape(conv, [])
<br>dense   = tf.layers.dense( flat, units= )


In [ ]:
def model_function(features, labels, mode):
  input_layer = tf.reshape(features["x"], [-1, W, H, D])
  ...
  dropout = tf.layers.dropout(dense, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
  logits  = tf.layers.dense( dropout, units=2 )#boxes or no boxes

  predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  loss = tf.losses.sparse_softmax_cross_entropy( labels = labels, logits=logits )
  tf.summary.scalar("loss", loss)

  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
      #optimizer = tf.train.AdamOptimizer(learning_rate=0.001, epsilon=0.01)
      gradients, variables = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
      optimize = optimizer.apply_gradients(zip(gradients, variables), global_step=tf.train.get_global_step())
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=optimize)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec( mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Training.. load the training set and call train on the classifier (estimator).

In [ ]:
loadImages("circs_n_rects_train/")
print("input data: "+str(np.shape(input_images)))
print("output data: "+str(np.shape(output_classes)))
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input_images},
    y=output_classes,
    batch_size=10,
    num_epochs=None,
    shuffle=True)



In [ ]:
estimator.train( input_fn=train_input_fn, steps=10000 )

Now let's evaluate the trained model with evaluation data that the model has never seen. <p>Remember that the model might just learn the specific instances of rectangles/circles instead of learning the general shape. By introducing new data, we verify that has not happened.

In [ ]:
loadImages("circs_n_rects_eval/")
print("Evaluating train data")
print("input data: "+str(np.shape(input_images)))
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x":input_images},
  y=output_classes,
  num_epochs=1,
  shuffle=False)
print("Image size: "+str(W)+", "+str(H))

In [ ]:
eval_results = estimator.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
a = cv2.imread("circs_n_rects_eval/rect.jpg",0)#1=color, 0=grayscale
input_images  = np.asarray([a], dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x":input_images},
  num_epochs=1,
  shuffle=False)
pred_results = estimator.predict(input_fn=eval_input_fn)
print(list(pred_results))

If you're not happy with the results, try the following:
<li>Add a pooling layer and another convolutional layer (capture features in another scale) 
<li>Add more filters
<li>Make the batch size small